(1) 모듈 로딩, 데이터 준비

In [2]:
# 모듈 로딩
from sklearn.datasets import fetch_openml

In [3]:
mnist = fetch_openml('mnist_784')   # sklearn.utils._bunch.Bunch

C:\ProgramData\anaconda3\envs\Torch_PY38\lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
mnist.data.shape, mnist.target.shape

((70000, 784), (70000,))

In [5]:
type(mnist.data), type(mnist.target)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [6]:
mnist.data.describe()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
count,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,...,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.0,70000.0,70000.0,70000.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.197414,0.099543,0.046629,0.016614,0.012957,0.001714,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.991206,4.256304,2.783732,1.561822,1.553796,0.320889,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,254.000000,253.000000,253.000000,254.000000,62.000000,0.0,0.0,0.0,0.0


(2) 데이터 전처리

In [9]:
# 데이터 전처리
featureDF = mnist.data
featureDF = featureDF / 255     # 스케일링 (MinMaxScaler 효과)

In [12]:
targetSR = mnist.target
targetSR.unique()

['5', '0', '4', '1', '9', '2', '3', '6', '7', '8']
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

(3) 기계학습용 데이터셋 준비

In [13]:
from sklearn.model_selection import train_test_split

In [26]:
# 학습용 / 테스트용 분리
x_train, x_test, y_train, y_test = train_test_split(featureDF, targetSR, test_size=0.15, random_state=42)
x_train.shape, y_train.shape, x_test.shape, y_test.shape    # DataFrame

((59500, 784), (59500,), (10500, 784), (10500,))

In [30]:
x_train_arr = x_train.values        # ndarray로 변환하는 방법!!
print(type(x_train_arr), x_train_arr.shape)

<class 'numpy.ndarray'> (59500, 784)


(4) 학습 진행

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [21]:
# 랜덤 시드 설정
torch.manual_seed(1)

In [45]:
x_train_t = torch.tensor(x_train.values, dtype=torch.float32)
x_test_t = torch.tensor(x_test.values, dtype=torch.float32)
y_train_t = torch.tensor(y_train.astype('int').values, dtype=torch.long)
y_test_t = torch.tensor(y_train.astype('int').values, dtype=torch.long)

In [47]:
# One-Hot-Encoding (y_train_t)
y_train_one_hot = torch.zeros(59500, 10)
y_train_one_hot.scatter_(1, y_train_t.unsqueeze(1), 1)
y_train_one_hot.view(-1, 10)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

In [48]:
# One-Hot-Encoding (y_test_t)
y_test_one_hot = torch.zeros(59500, 10)
y_test_one_hot.scatter_(1, y_test_t.unsqueeze(1), 1)
y_test_one_hot.view(-1, 10)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

In [57]:
model = nn.Sequential(
    nn.Linear(784, 10),
    nn.Softmax(dim=1)
)

In [63]:
optimizer = optim.SGD(model.parameters(), lr=1) # lr : 학습률 (learning rate)

In [64]:
# 학습 -----------------------------------------------
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train_t)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train_one_hot)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/1000 Cost: 1.581509
Epoch  100/1000 Cost: 1.577592
Epoch  200/1000 Cost: 1.574428
Epoch  300/1000 Cost: 1.571797
Epoch  400/1000 Cost: 1.569559
Epoch  500/1000 Cost: 1.567624
Epoch  600/1000 Cost: 1.565927
Epoch  700/1000 Cost: 1.564421
Epoch  800/1000 Cost: 1.563071
Epoch  900/1000 Cost: 1.561850
Epoch 1000/1000 Cost: 1.560738
